In [1]:
import sys
from pathlib import Path

sys.path.append(str(Path("../..").resolve()))

from src.data_ingestion import *
from src.data_preprocessing import *

from pyspark.sql import DataFrame
from pyspark.sql.functions import col
from pyspark.sql.types import NumericType, StringType
from pyspark.sql import functions as F

import seaborn as sns

import numpy as np

from itertools import combinations

from scipy import stats

import matplotlib.pyplot as plt

import pandas as pd
from pyspark.sql.window import Window

from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier




In [2]:
spark = init_spark()
df = load_data(spark, "../../data/US_Accidents_March23.csv")

In [18]:
df = preprocess_data(df)

Features with one unique value: []


In [4]:
df = df.withColumn("Hour", F.hour("Start_Time"))
df = df.withColumn("DayOfWeek", F.dayofweek("Start_Time"))  # Sunday = 1
df = df.withColumn("Month", F.month("Start_Time"))

In [17]:
df = df.drop("Start_Lat", "Start_Lng", "Start_Time", "End_Time")


In [6]:
# 4. Define categorical columns for encoding
categorical_cols = ["State", "Timezone", "Weather_Condition", "Sunrise_Sunset"]
indexers = [StringIndexer(inputCol=col, outputCol=col + "_Index", handleInvalid="keep") for col in categorical_cols]
encoders = [OneHotEncoder(inputCol=col + "_Index", outputCol=col + "_Vec") for col in categorical_cols]

# 5. Define numeric + boolean + derived time columns
numeric_cols = [
    "Distance(mi)", "Temperature(F)", "Humidity(%)", "Pressure(in)",
    "Visibility(mi)", "Wind_Speed(mph)", "Hour", "DayOfWeek", "Month"
]

boolean_cols = [
    "Amenity", "Bump", "Crossing", "Give_Way", "Junction", "No_Exit", "Railway",
    "Roundabout", "Station", "Stop", "Traffic_Calming", "Traffic_Signal"
]

# Fill null boolean columns with False (assumption)
for col in boolean_cols:
    df = df.withColumn(col, F.when(F.col(col).isNull(), F.lit(False)).otherwise(F.col(col).cast("boolean")))

# 6. Assemble all features into a vector
assembler_inputs = numeric_cols + boolean_cols + [col + "_Vec" for col in categorical_cols]
assembler = VectorAssembler(inputCols=assembler_inputs, outputCol="features")

# 7. Define the full pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler])

# 8. Fit and transform the data
model_ready_df = pipeline.fit(df).transform(df)

# 9. Select final dataset for modeling
final_df = model_ready_df.select("features", "Severity")

In [ ]:
train_df, test_df = final_df.randomSplit([0.8, 0.2], seed=42)

rf = RandomForestClassifier(labelCol="Severity", featuresCol="features")
rf_model = rf.fit(train_df)
predictions = rf_model.transform(test_df)

In [23]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Accuracy
accuracy_eval = MulticlassClassificationEvaluator(
    labelCol="Severity", predictionCol="prediction", metricName="accuracy"
)
accuracy = accuracy_eval.evaluate(predictions)

# F1 Score
f1_eval = MulticlassClassificationEvaluator(
    labelCol="Severity", predictionCol="prediction", metricName="f1"
)
f1 = f1_eval.evaluate(predictions)

# Precision
precision_eval = MulticlassClassificationEvaluator(
    labelCol="Severity", predictionCol="prediction", metricName="weightedPrecision"
)
precision = precision_eval.evaluate(predictions)

# Recall
recall_eval = MulticlassClassificationEvaluator(
    labelCol="Severity", predictionCol="prediction", metricName="weightedRecall"
)
recall = recall_eval.evaluate(predictions)

print(f"Accuracy:  {accuracy:.4f}")
print(f"F1 Score:  {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")


Accuracy:  0.7965
F1 Score:  0.7063
Precision: 0.6345
Recall:    0.7965


In [24]:
from pyspark.ml.classification import LogisticRegression, OneVsRest

# Define base classifier
lr = LogisticRegression(labelCol="Severity", featuresCol="features", maxIter=10)

# Wrap in One-vs-Rest
ovr = OneVsRest(classifier=lr, labelCol="Severity", featuresCol="features")
ovr_model = ovr.fit(train_df)

# Predictions
ovr_predictions = ovr_model.transform(test_df)


ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "c:\Users\Ahmed Osama\AppData\Local\Programs\Python\Python313\Lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ~~~~~~~~~~~~~~~~~~~~^^
  File "c:\Users\Ahmed Osama\AppData\Local\Programs\Python\Python313\Lib\socket.py", line 719, in readinto
    return self._sock.recv_into(b)
           ~~~~~~~~~~~~~~~~~~~~^^^
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\Ahmed Osama\AppData\Local\Programs\Python\Python313\Lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\Ahmed Osama\AppData\Local\Programs\Python\Python313\Lib\site-packages\py4j\clientserver.py", lin

ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Loop over each class to compute AUC individually
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction", metricName="areaUnderROC")

# Store AUCs
auc_scores = []

for i in range(4):  # assuming classes are [1, 2, 3, 4]
    bin_df = test_df.withColumn("label", F.when(F.col("Severity") == i, 1.0).otherwise(0.0))
    bin_model = LogisticRegression(labelCol="label", featuresCol="features").fit(train_df.withColumn("label", F.when(F.col("Severity") == i, 1.0).otherwise(0.0)))
    bin_pred = bin_model.transform(bin_df)
    auc = evaluator.evaluate(bin_pred)
    auc_scores.append((f"Class {i}", auc))

# Show AUC for each class
for cls, auc in auc_scores:
    print(f"{cls} AUC: {auc:.4f}")


# Binary Classification for Severity 

In [10]:
from pyspark.sql.functions import when

binary_df = final_df.withColumn(
    "label", when(F.col("Severity").isin(3, 4), 1).otherwise(0)
)
small_df = binary_df.sample(fraction=0.2, seed=42)
train_df, test_df = small_df.randomSplit([0.8, 0.2], seed=42)

In [11]:
from pyspark.ml.classification import (
    LogisticRegression, RandomForestClassifier,
    GBTClassifier, DecisionTreeClassifier
)

models = {
    "Logistic Regression": LogisticRegression(labelCol="label", featuresCol="features", maxIter=10),
    "Random Forest": RandomForestClassifier(labelCol="label", featuresCol="features", numTrees=20),
    "Gradient-Boosted Trees": GBTClassifier(labelCol="label", featuresCol="features", maxIter=20),
    "Decision Tree": DecisionTreeClassifier(labelCol="label", featuresCol="features", maxDepth=5),
}

# Train all models
fitted_models = {name: model.fit(train_df) for name, model in models.items()}
# Predict with each
predictions = {name: model.transform(test_df) for name, model in fitted_models.items()}


In [12]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

def evaluate_model(name, pred_df):
    evaluator_auc = BinaryClassificationEvaluator(labelCol="label", metricName="areaUnderROC")
    evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", metricName="f1")
    evaluator_acc = MulticlassClassificationEvaluator(labelCol="label", metricName="accuracy")

    auc = evaluator_auc.evaluate(pred_df)
    f1 = evaluator_f1.evaluate(pred_df)
    acc = evaluator_acc.evaluate(pred_df)

    return {"Model": name, "AUC": auc, "F1 Score": f1, "Accuracy": acc}

results = [evaluate_model(name, pred_df) for name, pred_df in predictions.items()]
for r in results:
    print(f"{r['Model']}: AUC={r['AUC']:.4f}, F1={r['F1 Score']:.4f}, Accuracy={r['Accuracy']:.4f}")


Logistic Regression: AUC=0.7297, F1=0.7492, Accuracy=0.8077
Random Forest: AUC=0.7509, F1=0.7184, Accuracy=0.8053
Gradient-Boosted Trees: AUC=0.8089, F1=0.7660, Accuracy=0.8192
Decision Tree: AUC=0.6452, F1=0.7438, Accuracy=0.8112


In [15]:
model_save_paths = {
    "Logistic Regression": "./logistic_regression_model",
    "Random Forest": "./random_forest_model",
    "Gradient-Boosted Trees": "./gbt_model",
    "Decision Tree": "./decision_tree_model",
}

for name, model in fitted_models.items():
    model.save(model_save_paths[name])


Py4JJavaError: An error occurred while calling o1981.save.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.jav